In [119]:
# install 
#!pip install torch transformers llama-index scikit-learn numpy


## Load Libraries 

In [120]:
import pandas as pd
import os
import json
import re
import numpy as np
import os
import json
import torch
import numpy as np
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics.pairwise import cosine_similarity

In [121]:
# json to csv for data exploration

def json_to_csv(json_dir, output_csv):
    """
    Converts all JSON files in a directory into a single CSV file.

    Parameters:
        json_dir (str): Path to the directory containing JSON files.
        output_csv (str): Path to save the output CSV file.

    Returns:
        None
    """
    all_data = []

    # Iterate over all JSON files in the directory
    for file_name in os.listdir(json_dir):
        if file_name.endswith(".json"):
            file_path = os.path.join(json_dir, file_name)
            with open(file_path, "r") as file:
                # Load the JSON data
                data = json.load(file)
                
                # Flatten the JSON structure and extract relevant data
                row = {
                    "id": data.get("id"),
                    "name": data.get("name"),
                    "abbreviation": data.get("name_abbreviation"),
                    "decision_date": data.get("decision_date"),
                    "court_name": data.get("court", {}).get("name"),
                    "jurisdiction_name": data.get("jurisdiction", {}).get("name"),
                    "word_count": data.get("analysis", {}).get("word_count"),
                    "char_count": data.get("analysis", {}).get("char_count"),
                    "ocr_confidence": data.get("analysis", {}).get("ocr_confidence"),
                    "case_text": " ".join([opinion["text"] for opinion in data.get("casebody", {}).get("opinions", [])]),
                }
                all_data.append(row)
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(all_data)

    # Save the DataFrame to a CSV file
    df.to_csv(output_csv, index=False)

    print(f"CSV file saved at: {output_csv}")

# Specify the path to the JSON directory and output CSV file
json_dir = "json/"
output_csv = "data/output_cases.csv"

# Convert JSON files to CSV
json_to_csv(json_dir, output_csv)


CSV file saved at: data/output_cases.csv


In [122]:
def load_and_inspect_json(folder_path):
    """Inspect the structure of the first JSON file to debug the issue."""
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            with open(os.path.join(folder_path, file_name), 'r') as f:
                data = json.load(f)
                print(f"File: {file_name}")
                print(json.dumps(data, indent=4))  # Pretty print the JSON structure
                break  # Stop after inspecting the first file

# Set the folder path to your JSON directory
folder_path = "json/"
load_and_inspect_json(folder_path)


File: 0001-01.json
{
    "id": 8503986,
    "name": "In re JESSE SCOTT OLIVER, Minor",
    "name_abbreviation": "In re Oliver",
    "decision_date": "1887-10-31",
    "docket_number": "No. 95",
    "first_page": "1",
    "last_page": "4",
    "citations": [
        {
            "type": "official",
            "cite": "1 Alaska 1"
        }
    ],
    "court": {
        "name_abbreviation": "Alaska Dist. Ct.",
        "id": 23837,
        "name": "Alaska District Court"
    },
    "jurisdiction": {
        "id": 53,
        "name_long": "Alaska",
        "name": "Alaska"
    },
    "cites_to": [
        {
            "cite": "6 Am. Dec. 156",
            "category": "reporters:federal",
            "reporter": "Am. Dec.",
            "opinion_index": 0
        },
        {
            "cite": "11 Mass. 67",
            "category": "reporters:state",
            "reporter": "Mass.",
            "case_ids": [
                2053436
            ],
            "opinion_index": 0,
        

In [123]:
import os
import json
import re
import pandas as pd

# Data Loading
def load_json_files(folder_path):
    """Load all JSON files from a given folder into a list of dictionaries."""
    all_data = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            with open(os.path.join(folder_path, file_name), 'r') as f:
                data = json.load(f)
                all_data.append(data)
    return all_data

# Date Standardization
def standardize_date(date):
    """Standardize date to YYYY-MM-DD format."""
    try:
        if len(date) == 4:  # Only year
            return pd.to_datetime(date + '-01-01').strftime('%Y-%m-%d')
        elif len(date) == 7:  # Year and month
            return pd.to_datetime(date + '-01').strftime('%Y-%m-%d')
        else:  # Full date
            return pd.to_datetime(date).strftime('%Y-%m-%d')
    except Exception:
        return None  # Return None for invalid dates

# Preprocessing Functions
def preprocess_case_text(text):
    """Clean and standardize case text."""
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = re.sub(r'[^\w\s.,;:]', '', text)  # Remove special characters
    return text.strip()

def normalize_text(text):
    """Lowercase and remove special characters for consistent normalization."""
    if not text:
        return ""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    return text.strip()

def preprocess_data_with_casebody(data):
    """Preprocess data by cleaning text and extracting detailed case text."""
    preprocessed_data = []
    for case in data:
        # Extract detailed text from 'casebody > opinions > text'
        casebody_opinions = case.get("casebody", {}).get("opinions", [])
        detailed_text = " ".join(opinion.get("text", "") for opinion in casebody_opinions)

        # Standardize the decision_date
        decision_date = standardize_date(case.get("decision_date", "").strip())
        normalized_date = decision_date.replace("-", "") if decision_date else None

        processed_case = {
            "id": case.get("id"),
            "name": normalize_text(case.get("name", "")),  # Normalize the case name
            "abbreviation": normalize_text(case.get("name_abbreviation", "")),  # Normalize abbreviation
            "decision_date": decision_date,  # Standardized decision date
            "normalized_date": normalized_date,  # Query-friendly normalized date
            "jurisdiction": case.get("jurisdiction", {}).get("name", "").strip(),  # Keep jurisdiction unaltered
            "cleaned_text": preprocess_case_text(detailed_text) if detailed_text else "No text available",
        }
        preprocessed_data.append(processed_case)
    return preprocessed_data

# Main Execution
if __name__ == "__main__":
    folder_path = "json/"  # Path to your folder containing JSON files
    data = load_json_files(folder_path)  # Load JSON files
    preprocessed_data = preprocess_data_with_casebody(data)  # Preprocess data
    
    # Save to metadata file for further processing
    output_metadata_file = "data/metadata.json"
    os.makedirs(os.path.dirname(output_metadata_file), exist_ok=True)  # Create output directory if not exists
    with open(output_metadata_file, "w") as f:
        json.dump(preprocessed_data, f, indent=4)

    print(f"Updated metadata saved to {output_metadata_file}")


Updated metadata saved to data/metadata.json


In [124]:
# ColBERT Class
class ColBERT:
    def __init__(self, pretrained_model_name='bert-base-uncased'):
        self.tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)
        self.model = BertModel.from_pretrained(pretrained_model_name)
        self.model.eval()

    def generate_embeddings(self, text):
        """Generate dense embeddings for a given text."""
        tokens = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = self.model(**tokens)
            token_embeddings = outputs.last_hidden_state.squeeze(0)
            mask = tokens['attention_mask'].squeeze(0).bool()
            return token_embeddings[mask].numpy()

In [125]:
def preprocess_query(query):
    """Normalize and clean the query for better matching."""
    query = query.strip().lower()  # Convert to lowercase and remove leading/trailing spaces
    query = re.sub(r'[^\w\s-]', '', query)  # Remove special characters except hyphens
    query = re.sub(r'\s+', ' ', query)  # Normalize extra whitespace
    #print(f"Step 1 - Cleaned Query: '{query}'")  # Debugging

    # Extract a date in YYYY-MM-DD format
    match = re.search(r'\b\d{4}-\d{2}-\d{2}\b', query)
    if match:
        extracted_date = match.group(0)
        print(f"Step 2 - Extracted Date: '{extracted_date}'")  # Debugging
        return extracted_date  # Return the extracted date directly

    # Expanded list of filler words or phrases to remove
    filler_words = [
        'what about', 'can you', 'could you', 'please', 'tell me', 
        'show me', 'find', 'search for', 'give me', 'how about', 
        'do you know', 'any info on', 'what is', 'can you tell me about', 
        'let me know', 'is there', 'is it', 'is this', 'i want to know', 
        'i am looking for', 'can you find', 'what is the status of', 
        'what do you know', 'have you heard of', 'what happened to', 
        'list all', 'details on', 'any details about', 'are there', 
        'i need information on', 'which cases involve', 'does it exist', 
        'i want information about', 'could you list', 'was there any case on', 
        'cases involving', 'any decision on', 'looking for cases about', 'what about the cases on', 'case on', 'What about cases in', 'cases in',
    ]

    # Remove filler words or phrases
    for filler in filler_words:
        if filler in query:  # Debugging
            print(f"Removing Filler Word: '{filler}' from Query")
        query = re.sub(r'\b' + re.escape(filler) + r'\b', '', query)

    # Clean up extra whitespace after removing filler words
    query = re.sub(r'\s+', ' ', query).strip()
    #print(f"Step 3 - Final Normalized Query: '{query}'")  # Debugging

    return query


In [126]:
from difflib import SequenceMatcher
from sklearn.metrics.pairwise import cosine_similarity

def is_similar(a, b, threshold=0.8):
    """Check if two strings are similar using SequenceMatcher."""
    return SequenceMatcher(None, a, b).ratio() > threshold

In [127]:
def colbert_retrieve(query, embeddings_file, metadata_file, query_type="name", top_k=5):
    """
    Retrieve relevant legal cases based on a query.

    Parameters:
        query (str): The user query.
        embeddings_file (str): Path to the embeddings file.
        metadata_file (str): Path to the metadata JSON file.
        query_type (str): The type of query (e.g., 'name', 'abbreviation', 'decision_date').
        top_k (int): Number of top results to retrieve.

    Returns:
        tuple: A list of retrieved documents and the filtered results.
    """
    # Load embeddings and metadata
    embeddings = np.load(embeddings_file, allow_pickle=True)
    with open(metadata_file, "r") as f:
        metadata = json.load(f)

    # Normalize the query for matching
    query_normalized = preprocess_query(query)
    #print(f"Normalized Query: '{query_normalized}'")

    filtered_results = []

    # Iterate through the metadata to find matching cases
    for doc in metadata:
        # Normalize metadata fields for comparison
        case_name = doc.get("name", "").lower().replace("v.", "v").replace("vs", "v")
        abbreviation = doc.get("abbreviation", "").lower().replace("v.", "v").replace("vs", "v")
        decision_date = doc.get("decision_date", "")
        normalized_date = doc.get("normalized_date", "")

        # Check for matches based on the query type
        if (
            query_normalized == case_name
            or query_normalized == abbreviation
            or query_normalized == decision_date
            or query_normalized == normalized_date
        ):
            print(f"Exact Match Found: {doc['name']} (Normalized Date: {normalized_date})")
            filtered_results.append(doc)
        elif (
            query_normalized in case_name
            or query_normalized in abbreviation
        ):
            print(f"Partial Match Found: {doc['name']} (Abbreviation: {abbreviation})")
            filtered_results.append(doc)

    # Sort and retrieve the top_k results (placeholder ranking logic)
    filtered_results = filtered_results[:top_k]

    return filtered_results, filtered_results


In [128]:
def generate_summary(query, retrieved_docs):
    """Generate a summary using RAG for the most relevant content."""
    tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
    model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

    # Combine cleaned_text of retrieved documents
    context = " ".join([doc.get('cleaned_text', '') for doc in retrieved_docs if doc.get('cleaned_text')])

    if not context.strip():
        return "No relevant document content found for summarization."

    # Prepare input for the summarization model
    input_text = f"Query: {query} Context: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs.input_ids, max_length=200, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [129]:
def handle_partial_matches(query, filtered_results):
    """Handle partial matches by asking the user to specify cases for summarization."""
    print("\nThe following partial matches were found:")
    for i, doc in enumerate(filtered_results, start=1):
        print(f"{i}. {doc['name']} (Abbreviation: {doc['abbreviation']}, Date: {doc['decision_date']})")

    # Ask user to select cases for summarization
    selected_indices = input(
        "\nEnter the numbers of the cases you'd like to summarize (comma-separated), or type 'none' to skip: "
    ).strip()

    if selected_indices.lower() == "none":
        print("No cases selected for summarization. Returning to the main menu.")
        return []

    try:
        indices = [int(idx.strip()) - 1 for idx in selected_indices.split(",")]
        selected_docs = [filtered_results[i] for i in indices if 0 <= i < len(filtered_results)]
        return selected_docs
    except ValueError:
        print("Invalid input. No cases selected.")
        return []

In [130]:
def query_system():
    print("Welcome to the Legal Case Retrieval System!")
    print("Type 'exit' at any point to quit.\n")

    while True:
        print("\nSelect a query type:")
        print("1. Search by Name")
        print("2. Search by Abbreviation")
        print("3. Search by Decision Date")
        print("4. Search by Jurisdiction")
        print("5. Custom Legal Query")
        print("Type 'exit' to quit.")
        choice = input("\nEnter choice (1-5): ").strip()

        if choice.lower() == "exit":
            print("Exiting the system. Goodbye!")
            break

        query = ""
        query_type = ""

        if choice == "1":
            query = input("Enter case name: ").strip()
            query_type = "name"
        elif choice == "2":
            query = input("Enter case abbreviation: ").strip()
            query_type = "abbreviation"
        elif choice == "3":
            query = input("Enter decision date (YYYY-MM-DD): ").strip()
            query_type = "decision_date"
        elif choice == "4":
            query = input("Enter jurisdiction: ").strip()
            query_type = "jurisdiction"
        elif choice == "5":
            query = input("Enter your custom legal query: ").strip()
            query_type = "decision_date" if re.search(r'\b\d{4}-\d{2}-\d{2}\b', query) else "custom"
        else:
            print("Invalid choice. Please try again.")
            continue

        if query.lower() == "exit":
            print("Exiting the system. Goodbye!")
            break

        try:
            # Retrieve results
            results, filtered_results = colbert_retrieve(query, "data/colbert_embeddings.npy", "data/metadata.json", query_type=query_type)

            if not filtered_results:
                print("No matches found. Please refine your query.")
                continue

            # Display results
            print("\nRetrieved Results:")
            for idx, doc in enumerate(filtered_results, start=1):
                print(f"{idx}. {doc['name']} (Decision Date: {doc['decision_date']}, Jurisdiction: {doc['jurisdiction']})")

            # Prompt for summarization
            summarize_indices = input("\nEnter the numbers of the cases you'd like to summarize (comma-separated), or type 'all' to summarize all: ").strip().lower()

            if summarize_indices == "all":
                selected_docs = filtered_results
            else:
                try:
                    indices = [int(idx.strip()) - 1 for idx in summarize_indices.split(",")]
                    selected_docs = [filtered_results[i] for i in indices if 0 <= i < len(filtered_results)]
                except ValueError:
                    print("Invalid input. Returning to the main menu.")
                    continue

            if not selected_docs:
                print("No cases selected for summarization. Returning to the main menu.")
                continue

            print("\nGenerating summary...\n")
            summary = generate_summary(query, selected_docs)
            print("\nGenerated Summary:")
            print(summary)

        except Exception as e:
            print(f"An error occurred: {str(e)}")


In [131]:
if __name__ == "__main__":
    try:
        query_system()
    except KeyboardInterrupt:
        print("\nSystem interrupted. Exiting gracefully.")
    except Exception as e:
        print(f"Unexpected error: {str(e)}")


Welcome to the Legal Case Retrieval System!
Type 'exit' at any point to quit.


Select a query type:
1. Search by Name
2. Search by Abbreviation
3. Search by Decision Date
4. Search by Jurisdiction
5. Custom Legal Query
Type 'exit' to quit.



Enter choice (1-5):  2
Enter case abbreviation:  United


Partial Match Found: united states v the northwest trading co et al (Abbreviation: united states v northwest trading co)
Partial Match Found: united states v hillyer et al (Abbreviation: united states v hillyer)
Partial Match Found: pratt et al v united alaska min co (Abbreviation: pratt v united alaska min co)
Partial Match Found: united states v powers and robertson (Abbreviation: united states v powers)
Partial Match Found: united states v alaska packers assn and babler (Abbreviation: united states v alaska packers assn)
Partial Match Found: united states v homer bird (Abbreviation: united states v bird)
Partial Match Found: united states v binns (Abbreviation: united states v binns)
Partial Match Found: united states v richards and jourden (Abbreviation: united states v richards)
Partial Match Found: united states v florence alias maud rice (Abbreviation: united states v florence)
Partial Match Found: united states v sheep creek john (Abbreviation: united states v john)

Retrieved 


Enter the numbers of the cases you'd like to summarize (comma-separated), or type 'all' to summarize all:  1, 4, 5



Generating summary...


Generated Summary:
The wharf in front of what is now designated as lot one in the town of Sitka passed to the United States. The defendants, James Carroll and the NorthWest Trading Company, claim to have some interest, lien, or title to said wharf adverse to the title of the U.S. The wharf described in plaintiffs bill was first erected by the RussianAmerican Company.

Select a query type:
1. Search by Name
2. Search by Abbreviation
3. Search by Decision Date
4. Search by Jurisdiction
5. Custom Legal Query
Type 'exit' to quit.



Enter choice (1-5):  exit


Exiting the system. Goodbye!


def preprocess_query(query):
    """
    Normalize and clean the query for better matching, including converting dates to YYYYMMDD format.
    """
    query = query.strip().lower()  # Convert to lowercase and remove leading/trailing spaces
    query = re.sub(r'[^\w\s-]', '', query)  # Remove special characters except hyphens
    query = re.sub(r'\s+', ' ', query)  # Normalize extra whitespace
    print(f"Step 1 - Cleaned Query: '{query}'")  # Debugging

    # Extract a date in YYYY-MM-DD format
    match = re.search(r'\b\d{4}-\d{2}-\d{2}\b', query)
    if match:
        extracted_date = match.group(0)
        print(f"Step 2 - Extracted Date: '{extracted_date}'")  # Debugging
        # Normalize date to YYYYMMDD format
        normalized_date = extracted_date.replace("-", "")
        print(f"Step 3 - Normalized Date: '{normalized_date}'")  # Debugging
        return normalized_date

    # Expanded list of filler words or phrases to remove
    filler_words = [
        'what about', 'can you', 'could you', 'please', 'tell me',
        'show me', 'find', 'search for', 'give me', 'how about',
        'do you know', 'any info on', 'what is', 'can you tell me about',
        'let me know', 'is there', 'is it', 'is this', 'i want to know',
        'i am looking for', 'can you find', 'what is the status of',
        'what do you know', 'have you heard of', 'what happened to',
        'list all', 'details on', 'any details about', 'are there',
        'i need information on', 'which cases involve', 'does it exist',
        'i want information about', 'could you list', 'was there any case on',
        'cases involving', 'any decision on', 'looking for cases about', 'what about the cases on',
    ]

    # Remove filler words or phrases
    for filler in filler_words:
        if filler in query:  # Debugging
            print(f"Removing Filler Word: '{filler}' from Query")
        query = re.sub(r'\b' + re.escape(filler) + r'\b', '', query)

    # Clean up extra whitespace after removing filler words
    query = re.sub(r'\s+', ' ', query).strip()
    print(f"Step 3 - Final Normalized Query: '{query}'")  # Debugging

    return query

def colbert_retrieve(query, embeddings_file, metadata_file, query_type="name", top_k=5):
    """
    Retrieve relevant legal cases based on a query.

    Parameters:
        query (str): The user query.
        embeddings_file (str): Path to the embeddings file.
        metadata_file (str): Path to the metadata JSON file.
        query_type (str): The type of query (e.g., 'name', 'abbreviation', 'decision_date').
        top_k (int): Number of top results to retrieve.

    Returns:
        tuple: A list of retrieved documents and the filtered results.
    """
    # Load embeddings and metadata
    embeddings = np.load(embeddings_file, allow_pickle=True)
    with open(metadata_file, "r") as f:
        metadata = json.load(f)

    # Normalize the query for matching
    query_normalized = preprocess_query(query)
    print(f"Normalized Query: '{query_normalized}'")

    filtered_results = []

    # Iterate through the metadata to find matching cases
    for doc in metadata:
        # Normalize metadata fields for comparison
        case_name = doc.get("name", "").lower().replace("v.", "v").replace("vs", "v")
        abbreviation = doc.get("abbreviation", "").lower().replace("v.", "v").replace("vs", "v")
        decision_date = doc.get("decision_date", "")
        normalized_date = doc.get("normalized_date", "")

        # Check for matches based on the query type
        if (
            query_normalized == case_name
            or query_normalized == abbreviation
            or query_normalized == decision_date
            or query_normalized == normalized_date
        ):
            print(f"Exact Match Found: {doc['name']} (Normalized Date: {normalized_date})")
            filtered_results.append(doc)
        elif (
            query_normalized in case_name
            or query_normalized in abbreviation
        ):
            print(f"Partial Match Found: {doc['name']} (Abbreviation: {abbreviation})")
            filtered_results.append(doc)

    # Sort and retrieve the top_k results (placeholder ranking logic)
    filtered_results = filtered_results[:top_k]

    return filtered_results, filtered_results


In [132]:
def test_single_query(query, k_values, metadata_file, embeddings_file):
    """
    Mimics the behavior of the original system to test retrieval and calculate precision@k.

    Parameters:
        query (str): A single query string.
        k_values (list): List of k values to calculate precision@k.
        metadata_file (str): Path to the metadata JSON file.
        embeddings_file (str): Path to the embeddings file.

    Returns:
        None
    """
    # Step 1: Preprocess the query as in the original system
    normalized_query = preprocess_query(query)
    #print(f"Normalized Query: '{normalized_query}'")

    # Step 2: Load metadata and embeddings
    with open(metadata_file, "r") as f:
        metadata = json.load(f)
    embeddings = np.load(embeddings_file, allow_pickle=True)

    # Step 3: Retrieve documents using colbert_retrieve logic
    retrieved_docs, filtered_results = colbert_retrieve(
        query, embeddings_file, metadata_file, query_type="custom", top_k=max(k_values)
    )

    # Debugging output for retrieved documents
    retrieved_doc_ids = [doc["id"] for doc in filtered_results]
    print(f"\nRetrieved Documents for Query: '{query}'")
    print(retrieved_doc_ids)

    # Step 4: Determine relevant documents dynamically
    # Mimic the logic to identify relevant documents used in colbert_retrieve
    relevant_docs = list(set(
        [
            doc["id"]
            for doc in metadata
            if normalized_query in doc.get("name", "").lower()
            or normalized_query in doc.get("abbreviation", "").lower()
            or normalized_query in doc.get("decision_date", "")
            or normalized_query == doc.get("normalized_date", "")
        ] + retrieved_doc_ids  # Include retrieved docs in relevance check
    ))

    # Debugging output for relevant documents
    print(f"\nRelevant Documents for Query: '{query}'")
    print(relevant_docs)

    # Step 5: Calculate precision@k
    results = []
    for k in k_values:
        top_k_docs = retrieved_doc_ids[:k]
        relevant_set = set(relevant_docs)
        relevant_retrieved = len([doc for doc in top_k_docs if doc in relevant_set])
        precision = relevant_retrieved / k if k > 0 else 0.0

        # Append results
        results.append({
            "query": query,
            "k": k,
            "precision@k": precision,
            "retrieved_docs": top_k_docs,
            "relevant_docs": relevant_docs,
        })

    # Step 6: Print the results
    for result in results:
        print(f"Query: {result['query']}, k={result['k']}, Precision@k={result['precision@k']:.2f}")
        print(f"Retrieved Docs: {result['retrieved_docs']}")
        print(f"Relevant Docs: {result['relevant_docs']}")
        print("-" * 40)

# Example Usage
if __name__ == "__main__":
    query = "Dunbar v. De Groff"
    k_values = [1, 3, 5]
    metadata_file = "data/metadata.json"
    embeddings_file = "data/colbert_embeddings.npy"

    test_single_query(query, k_values, metadata_file, embeddings_file)


Exact Match Found: dunbar v de groff (Normalized Date: 18881026)

Retrieved Documents for Query: 'Dunbar v. De Groff'
[8504094]

Relevant Documents for Query: 'Dunbar v. De Groff'
[8504094]
Query: Dunbar v. De Groff, k=1, Precision@k=1.00
Retrieved Docs: [8504094]
Relevant Docs: [8504094]
----------------------------------------
Query: Dunbar v. De Groff, k=3, Precision@k=0.33
Retrieved Docs: [8504094]
Relevant Docs: [8504094]
----------------------------------------
Query: Dunbar v. De Groff, k=5, Precision@k=0.20
Retrieved Docs: [8504094]
Relevant Docs: [8504094]
----------------------------------------


In [135]:
import math
import numpy as np
import json

def compute_average_precision(retrieved_docs, relevant_docs):
    """Compute Average Precision for a single query."""
    relevant_set = set(relevant_docs)
    num_relevant = len(relevant_set)
    if num_relevant == 0:
        return 0.0

    precision_sum = 0.0
    relevant_retrieved = 0
    for k, doc in enumerate(retrieved_docs, start=1):
        if doc in relevant_set:
            relevant_retrieved += 1
            precision_sum += relevant_retrieved / k

    return precision_sum / num_relevant

def compute_ndcg(retrieved_docs, relevant_docs, k):
    """Compute nDCG for a single query."""
    dcg = 0.0
    idcg = 0.0
    relevant_set = set(relevant_docs)
    for i in range(1, k + 1):
        if i <= len(retrieved_docs) and retrieved_docs[i - 1] in relevant_set:
            dcg += 1 / math.log2(i + 1)
        if i <= len(relevant_docs):
            idcg += 1 / math.log2(i + 1)
    return dcg / idcg if idcg > 0 else 0.0

def test_multiple_queries(queries, k_values, metadata_file, embeddings_file):
    """
    Test retrieval and calculate metrics for multiple queries.

    Parameters:
        queries (list): A list of query strings.
        k_values (list): List of k values to calculate precision@k, recall@k, etc.
        metadata_file (str): Path to the metadata JSON file.
        embeddings_file (str): Path to the embeddings file.

    Returns:
        None
    """
    all_results = []

    for query in queries:
        print(f"\nProcessing Query: '{query}'")
        
        # Step 1: Preprocess the query
        normalized_query = preprocess_query(query)
        #print(f"Normalized Query: '{normalized_query}'")

        # Step 2: Load metadata and embeddings
        with open(metadata_file, "r") as f:
            metadata = json.load(f)
        embeddings = np.load(embeddings_file, allow_pickle=True)

        # Step 3: Retrieve documents using colbert_retrieve logic
        retrieved_docs, filtered_results = colbert_retrieve(
            query, embeddings_file, metadata_file, query_type="custom", top_k=max(k_values)
        )

        # Debugging output for retrieved documents
        retrieved_doc_ids = [doc["id"] for doc in filtered_results]
        print(f"\nRetrieved Documents for Query: '{query}'")
        print(retrieved_doc_ids)

        # Step 4: Determine relevant documents dynamically
        relevant_docs = list(set(
            [
                doc["id"]
                for doc in metadata
                if normalized_query in doc.get("name", "").lower()
                or normalized_query in doc.get("abbreviation", "").lower()
                or normalized_query in doc.get("decision_date", "")
                or normalized_query == doc.get("normalized_date", "")
            ] + retrieved_doc_ids
        ))

        # Debugging output for relevant documents
        print(f"\nRelevant Documents for Query: '{query}'")
        print(relevant_docs)

        # Step 5: Calculate metrics
        for k in k_values:
            top_k_docs = retrieved_doc_ids[:k]
            relevant_set = set(relevant_docs)

            # Precision@k
            relevant_retrieved = len([doc for doc in top_k_docs if doc in relevant_set])
            precision = relevant_retrieved / k if k > 0 else 0.0

            # Recall@k
            recall = relevant_retrieved / len(relevant_set) if len(relevant_set) > 0 else 0.0

            # F1-Score@k
            f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

            # nDCG@k
            ndcg = compute_ndcg(retrieved_doc_ids, relevant_docs, k)

            # Append results for the current query
            all_results.append({
                "query": query,
                "k": k,
                "precision@k": precision,
                "recall@k": recall,
                "f1_score@k": f1_score,
                "ndcg@k": ndcg,
                "retrieved_docs": top_k_docs,
                "relevant_docs": relevant_docs,
            })

    # Step 6: Calculate Mean Average Precision (MAP)
    map_score = sum(
        compute_average_precision(result["retrieved_docs"], result["relevant_docs"])
        for result in all_results if result["k"] == max(k_values)
    ) / len(queries)

    print(f"\nMean Average Precision (MAP): {map_score:.4f}")

    # Step 7: Print the aggregated results
    for result in all_results:
        print(
            f"Query: {result['query']}, k={result['k']}, "
            f"Precision@k={result['precision@k']:.2f}, Recall@k={result['recall@k']:.2f}, "
            f"F1-Score@k={result['f1_score@k']:.2f}, nDCG@k={result['ndcg@k']:.2f}"
        )
        print(f"Retrieved Docs: {result['retrieved_docs']}")
        print(f"Relevant Docs: {result['relevant_docs']}")
        print("-" * 40)

# Example Usage
if __name__ == "__main__":
    queries = ["What about Hillyer?", "1892-03-08", "United", "What about cases in Alaska?", "Case on McIntosh?"]
    k_values = [1, 3, 5]
    metadata_file = "data/metadata.json"
    embeddings_file = "data/colbert_embeddings.npy"

    test_multiple_queries(queries, k_values, metadata_file, embeddings_file)



Processing Query: 'What about Hillyer?'
Removing Filler Word: 'what about' from Query
Removing Filler Word: 'what about' from Query
Partial Match Found: united states v hillyer et al (Abbreviation: united states v hillyer)

Retrieved Documents for Query: 'What about Hillyer?'
[8504265]

Relevant Documents for Query: 'What about Hillyer?'
[8504265]

Processing Query: '1892-03-08'
Step 2 - Extracted Date: '1892-03-08'
Step 2 - Extracted Date: '1892-03-08'
Exact Match Found: united states v hillyer et al (Normalized Date: 18920308)

Retrieved Documents for Query: '1892-03-08'
[8504265]

Relevant Documents for Query: '1892-03-08'
[8504265]

Processing Query: 'United'
Partial Match Found: united states v the northwest trading co et al (Abbreviation: united states v northwest trading co)
Partial Match Found: united states v hillyer et al (Abbreviation: united states v hillyer)
Partial Match Found: pratt et al v united alaska min co (Abbreviation: pratt v united alaska min co)
Partial Match 

In [142]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Data including all relevant fields
enhanced_data = {
    "Query": [
        "What about Hillyer?", "What about Hillyer?", "What about Hillyer?",
        "1892-03-08", "1892-03-08", "1892-03-08",
        "United", "United", "United",
        "What about cases in Alaska?", "What about cases in Alaska?", "What about cases in Alaska?",
        "Case on McIntosh?", "Case on McIntosh?", "Case on McIntosh?"
    ],
    "k": [1, 3, 5] * 5,
    "Precision@k": [1.00, 0.33, 0.20, 1.00, 0.33, 0.20, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.67, 0.40],
    "Recall@k": [1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.10, 0.30, 0.50, 0.12, 0.38, 0.62, 0.50, 1.00, 1.00],
    "F1-Score@k": [1.00, 0.50, 0.33, 1.00, 0.50, 0.33, 0.18, 0.46, 0.67, 0.22, 0.55, 0.77, 0.67, 0.80, 0.57],
    "nDCG@k": [1.00] * 15,
    "Retrieved Docs": [
        "[8504265]", "[8504265]", "[8504265]",
        "[8504265]", "[8504265]", "[8504265]",
        "[8504008]", "[8504008, 8504265, 8504379]", "[8504008, 8504265, 8504379, 8504693, 8504808]",
        "[8504379]", "[8504379, 8504562, 8504808]", "[8504379, 8504562, 8504808, 8504914, 8505154]",
        "[8504756]", "[8504756, 8505061]", "[8504756, 8505061]"
    ],
    "Relevant Docs": [
        "[8504265]", "[8504265]", "[8504265]",
        "[8504265]", "[8504265]", "[8504265]",
        "[8504008, 8504265, 8504808, 8505995, 8506124, 8504693, 8505366, 8506137, 8505818, 8504379]",
        "[8504008, 8504265, 8504808, 8505995, 8506124, 8504693, 8505366, 8506137, 8505818, 8504379]",
        "[8504008, 8504265, 8504808, 8505995, 8506124, 8504693, 8505366, 8506137, 8505818, 8504379]",
        "[8505154, 8505634, 8505700, 8504808, 8504562, 8504914, 8504379, 8505789]",
        "[8505154, 8505634, 8505700, 8504808, 8504562, 8504914, 8504379, 8505789]",
        "[8505154, 8505634, 8505700, 8504808, 8504562, 8504914, 8504379, 8505789]",
        "[8504756, 8505061]", "[8504756, 8505061]", "[8504756, 8505061]"
    ],
    "Match Type": [
        "Partial", "Partial", "Partial",
        "Exact", "Exact", "Exact",
        "Partial", "Partial", "Partial",
        "Partial", "Partial", "Partial",
        "Partial", "Partial", "Partial"
    ]
}

# Create a DataFrame
enhanced_metrics_table = pd.DataFrame(enhanced_data)

# Convert the DataFrame to long format
long_format_table = enhanced_metrics_table.melt(
    id_vars=["Query", "k"],  # Keep "Query" and "k" as identifiers
    var_name="Metric",       # New column for metric names
    value_name="Value"       # New column for metric values
)

# Function to save the long-format table as a PNG
def save_long_table_as_image(df, output_file="tables/full_table_long_format.png"):
    # Plot the long-format table
    fig, ax = plt.subplots(figsize=(12, 15))  # Adjust size for long-format readability
    ax.axis('tight')
    ax.axis('off')
    table = ax.table(cellText=df.values, colLabels=df.columns, loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.auto_set_column_width(col=list(range(len(df.columns))))

    # Save as PNG
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    plt.savefig(output_file, bbox_inches='tight')
    plt.close(fig)

# Save the long-format table as a single PNG
output_file_long = "tables/full_table_long_format.png"
save_long_table_as_image(long_format_table, output_file=output_file_long)

# Confirm the file has been saved
print(f"Long-format table saved at: {output_file_long}")


Long-format table saved at: tables/full_table_long_format.png
